In [1]:
using Yao
using FFTW, Test
using LinearAlgebra: I

In [2]:
# Control-R(k) gate in block-A
A(i::Int, j::Int, k::Int) = control([i, ], j=>shift(2π/(1<<k)))
# block-B
B(n::Int, i::Int) = chain(i==j ? put(i=>H) : A(j, i, j-i+1) for j = i:n)
QFT(n::Int) = chain(n, B(n, i) for i = 1:n)

QFT (generic function with 1 method)

In [3]:
# define QFT and IQFT block.
num_bit = 5
qft = QFT(num_bit)
iqft = qft'

nqubits: 5
chain
├─ chain
│  └─ put on (5)
│     └─ H
├─ chain
│  ├─ control(5)
│  │  └─ (4,) shift(-1.5707963267948966)
│  └─ put on (4)
│     └─ H
├─ chain
│  ├─ control(5)
│  │  └─ (3,) shift(-0.7853981633974483)
│  ├─ control(4)
│  │  └─ (3,) shift(-1.5707963267948966)
│  └─ put on (3)
│     └─ H
├─ chain
│  ├─ control(5)
│  │  └─ (2,) shift(-0.39269908169872414)
│  ├─ control(4)
│  │  └─ (2,) shift(-0.7853981633974483)
│  ├─ control(3)
│  │  └─ (2,) shift(-1.5707963267948966)
│  └─ put on (2)
│     └─ H
└─ chain
   ├─ control(5)
   │  └─ (1,) shift(-0.19634954084936207)
   ├─ control(4)
   │  └─ (1,) shift(-0.39269908169872414)
   ├─ control(3)
   │  └─ (1,) shift(-0.7853981633974483)
   ├─ control(2)
   │  └─ (1,) shift(-1.5707963267948966)
   └─ put on (1)
      └─ H


In [4]:
@test chain(num_bit, qft, iqft) |> mat ≈ I

Test Passed

In [5]:
# define a register and get its vector representation
reg = rand_state(num_bit)
rv = reg |> statevec |> copy

32-element Array{Complex{Float64},1}:
    0.03705176264223667 + 0.07030158690307886im
   -0.06493281009430363 + 0.031000617317167127im
    0.02335638671030601 + 0.019114569528695835im
   -0.03833297350889918 + 0.019960964407197693im
     0.3325235683528705 + 0.0043371507926670575im
   -0.08558421609328952 + 0.23013305591893182im
    0.04444952877132521 - 0.15040260530916286im
     0.2116426182903454 - 0.034687013429928096im
    -0.3019160059678119 - 0.04412890641267696im
   -0.15755670670518498 + 0.011480824161192471im
    0.08113386527491738 + 0.13247728763076158im
    -0.1724713637245056 + 0.32747854377382934im
    -0.0850955999017533 + 0.23427392189521354im
                        ⋮
    0.11833239905401066 - 0.029675122362028464im
  -0.013428018110239514 - 0.06923936189691048im
    0.04838599006519841 + 0.07001201489304615im
    0.06155631141063008 - 0.013101548081074213im
    0.17498523248931214 - 0.014251017141044719im
    -0.1049315501933095 - 0.16490085209430294im
 -0.0041748957

In [10]:
# test fft
reg_qft = copy(reg) |> invorder! |> qft
kv = ifft(rv)*sqrt(length(rv))
@test reg_qft |> statevec ≈ kv

Test Passed

In [13]:
# test ifft
reg_iqft = copy(reg) |>iqft
kv = fft(rv)/sqrt(length(rv))

32-element Array{Complex{Float64},1}:
   0.058190699461627186 + 0.08542111795583401im
    0.16628896772457744 - 0.010833839739424572im
   0.039866392874092604 - 0.1524475742636332im
    0.05146373373014503 + 0.11860107344730701im
    0.10302935053604129 + 0.02995458865748594im
   -0.16331075831081462 + 0.12908035356381084im
    0.04976517452553545 + 0.24112223772925911im
   -0.08400735531406955 + 0.0065397545790106424im
   -0.08696263370704814 + 0.1584616853827835im
    0.08582154305767468 - 0.006891507052186565im
    0.11522004215030845 - 0.22902746585350553im
   -0.12480221801615386 - 0.16509644984830352im
   -0.12367835272775465 + 0.16734062622828771im
                        ⋮
   -0.22414765685136845 - 0.029986625189387175im
    0.10619522195307357 + 0.05460858098882705im
  -0.011761447816434695 + 0.09464920346599219im
    0.11886797686373077 - 0.19263875118112062im
    0.03929985058015911 - 0.027822589974935745im
    0.04206064792322077 + 0.08814945011582993im
 -0.0058179205968365